In [6]:
# IMPORTSs
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from dateutil import parser


In [12]:
# LENDO O DATAFRAME
df = pd.read_csv('steam_games.csv')

In [13]:
print(df.shape)
print(df.isna().sum())

(40833, 20)
url                             0
types                           2
name                           16
desc_snippet                13221
recent_reviews              38127
all_reviews                 12363
release_date                 3179
developer                     343
publisher                    5100
popular_tags                 2945
game_details                  520
languages                      36
achievements                28639
genre                         438
game_description             2913
mature_content              37936
minimum_requirements        19764
recommended_requirements    19758
original_price               5311
discount_price              26290
dtype: int64


In [14]:
# colunas que irei excluir 
df = df.drop(columns=['recommended_requirements', 'minimum_requirements',
                      'mature_content', 'recent_reviews', 'desc_snippet'])



In [44]:
# funçoes
# transformando os valores numericos que estão como string em float e mantendo as urls que já estão
def convertendo_str_para_float_mantendo_as_url(valor):
    try:
        return float(valor)
    except ValueError:
        return valor
    
# pegando apenas string q tenha 'Free' ou 'free' ou 'R$' e mantendo os os valores numericos antigos
# os valores dos jogos obtidos após o webscrapitng veio nesse formato (R$ valor do jogo)
# assim filtrei esses valores dos valores confusos q vinheram
def mantendo_strings_estrategicas(valor):
    if isinstance(valor, str):
        if 'R$' in valor or 'Free' in valor or 'free' in valor:
            return valor
        else:
            return np.nan
    elif isinstance(valor, (float, int)):
        return valor
    else:
        return np.nan
    
#deixando os numeros no mesmo formato (valor.decimal) e trocando 'R$' -> '' e ',' -> '.'
# 'R$ 20,99'->'20.99'
def formatando_os_valores(valor):
    if isinstance(valor, str) and 'R$' in valor:
        valor = valor.replace('R$', '').replace(',', '.').strip()
        return float(valor)
    else:
        return valor
    
# transformando os valores em datetime e os valores que nao é possivel transformar vai virar um nan
# se a data não tiver um dia definido (may 2002)
# vai se considerado (may 1 2002)
def transformando_datas_em_datatime(text):
    for fmt in ('%b %d, %Y', '%b %Y', '%d/%m/%Y'):
        try:
            return pd.to_datetime(text, format=fmt)
        except ValueError:
            pass
    return np.nan

def recortando_a_lista_mantendo_url(text):
    if 'http' in text:
        return text  # Mantém URLs inalteradas
    return text.split('- ')[1].strip()


# tratando coluna original_price -> price

In [15]:
#transformando as linhas que estão com algum indicativo que o jogo seja gratuito 
# 'Free', 'Free To Play', 'Free to Play' ou 'Play for Free!' em 0
df['original_price'] = df['original_price'].apply(lambda x: 0 if x == 'Free' else x)
df['original_price'] = df['original_price'].apply(lambda x: 0 if x == 'Free To Play' else x)
df['original_price'] = df['original_price'].apply(lambda x: 0 if x == 'Free to Play' else x)
df['original_price'] = df['original_price'].apply(lambda x: 0 if x == 'Play for Free!' else x)

#transformando todos o valores em string
df['original_price'] = df['original_price'].astype(str)
# substituindo '$' --> ''
df['original_price'] = df['original_price'].str.replace('$', '')

# Criando uma nova coluna 'preco' com o valores numericos da coluna 'orginal_price' e as string serão NaN
df['preço'] = pd.to_numeric(df['original_price'], errors='coerce')

# Transformando os nan na url
df['preço'].fillna(df['url'], inplace=True)
lista_coluna_preco = list(df['preço'])

# pegando as URl
lista_com_url_coluna_preco = [x for x in lista_coluna_preco if isinstance(x, str)]

C:\Users\Wellington\AppData\Local\Temp\ipykernel_12096\1446868490.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['original_price'] = df['original_price'].str.replace('$', '')


In [16]:
#lista onde vou colocar os valores dos jogos
preco_dos_jogos_encontrados = []
# lista com as urls manipulaveis
url_utilizavel = []
#lista com url sem uso
urls_sem_uso = []

url = lista_com_url_coluna_preco

# criando um for para verificar cada url se da para usar ou nao
for i in url:
        
    response = requests.get(i)

    # Verificar se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Parse a página HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # indo na class que contem o preço 
        price_element = soup.find('div', {'class': 'game_purchase_price'})

        # vendo se tem valor na class
        if price_element:
            # pegando o valor que está dentro da class
            price_text = price_element.get_text().strip()
            preco_dos_jogos_encontrados.append(price_text)
            url_utilizavel.append(i)
        else:
            urls_sem_uso.append(i)
    else:
        urls_sem_uso.append(i)


In [18]:
# criando uma lista com os valores convertidos e as urls  
data_convertida = [convertendo_str_para_float_mantendo_as_url(valor) for valor in df['preço']]

# criando uma nova coluna com os valores convertidos e as urls
df['preço teste'] = data_convertida

In [20]:
#transformando as urls sem uso em nan
df.loc[df['preço teste'].isin(urls_sem_uso), 'preço teste'] = pd.NA

#excluindo os nan da coluna
df = df.dropna(subset=['preço teste'])

In [21]:
df = df.reset_index()

# Inicialize um contador
contador = 0

# Loop para substituir as strings
for i in range(len(df)):
    if isinstance(df.at[i, 'preço teste'], str) and contador < len(preco_dos_jogos_encontrados):
        df.at[i, 'preço teste'] = preco_dos_jogos_encontrados[contador]
        contador += 1

In [27]:
# coversao de dolar ----> real no atual momento 01/10/2023 o dolar está a 5.04
# Estou convertendo agora pq temos valores q acabaram de chegar do scraping e os antigos(q são float e estão em dolar)
df['price'] = df['preço teste'].apply(lambda x: round(x * 5.04, 2) if isinstance(x, (int, float)) else x)


In [29]:
# removendo um outlier
df_remove = df.loc[(df['price']=='R$ 1.118,88')]
df = df.drop(df_remove.index)

In [31]:
# criando coluna 'price' mantendo as string 'R$' 'free' 'Free'
df['price'] = df['preço teste'].apply(mantendo_strings_estrategicas)

In [32]:
# atualizando a coluna
df['price'] = df['price'].apply(formatando_os_valores)

In [33]:
# linhas com 'Free', 'Free To Play' e 'Free to Play' transformei em 0
df['price'] = df['price'].apply(lambda x: 0 if x == 'Free' else x)
df['price'] = df['price'].apply(lambda x: 0 if x == 'Free To Play' else x)
df['price'] = df['price'].apply(lambda x: 0 if x == 'Free to Play' else x)

In [34]:
print(df.shape)
print(df.isna().sum())

(36945, 19)
index                   0
url                     0
types                   1
name                    1
all_reviews          8926
release_date          751
developer             253
publisher            4818
popular_tags          627
game_details          458
languages              20
achievements        24820
genre                 357
game_description      590
original_price          0
discount_price      24736
preço                   0
preço teste             0
price                 260
dtype: int64


In [36]:
# transformando todas as strings em nan
df['price'] = df['price'].apply(lambda x: np.nan if isinstance(x, str) else x)

df = df.dropna(subset=['price'])
df = df.drop(columns=['preço teste', 'preço', 'original_price'])

In [37]:
print(df.shape)
print(df.isna().sum())

(36646, 16)
index                   0
url                     0
types                   1
name                    1
all_reviews          8840
release_date          743
developer             253
publisher            4816
popular_tags          627
game_details          453
languages              20
achievements        24652
genre                 356
game_description      590
discount_price      24486
price                   0
dtype: int64


# tratanado coluna discount_price -> desconto 

In [38]:
# tirando os '$' dos preços
df['discount_price'] = df['discount_price'].str.replace('$', '')
#transformando os valores em float
df['desconto'] = pd.to_numeric(df['discount_price'], errors='coerce')
# transformando nan em 0
df['desconto'] = df['desconto'].fillna(0)
# coversao de dolar ----> real no atual momento 01/10/2023 o dolar está a 5.04
df['desconto'] = df['desconto'].apply(lambda x: round(x * 5.04, 2) if isinstance(x, (int, float)) else x)

df = df.drop(columns=['discount_price'])

C:\Users\Wellington\AppData\Local\Temp\ipykernel_12096\2573350738.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['discount_price'] = df['discount_price'].str.replace('$', '')


# tratamento coluna game_description

In [39]:
# excluindo os valores nulos da coluna 
df = df.dropna(subset=['game_description'])

In [40]:
# pegando os valores unicos da coluna game_description
valores_unicos_da_coluna_game_description = list(df['game_description'].unique())

#verificando as descrições que tem uma qunatidade baixa de index
strings_pequenas = [st for st in valores_unicos_da_coluna_game_description if len(st) < 50]
# strings_pequenas

In [41]:
# TRANSFORMANDO UMA LINHA Q NAO TINHA NENHUMA DESCRIÇÃO EM NaN
df['game_description'] = df['game_description'].apply(lambda x: np.nan if x == ' About This Game  ' else x)
df['game_description'] = df['game_description'].apply(lambda x: np.nan if x == ' About This Game Null' else x)
df['game_description'] = df['game_description'].apply(lambda x: np.nan if x == ' About This Content  ' else x)
df['game_description'] = df['game_description'].apply(lambda x: np.nan if x == ' About This Software  ' else x)
df['game_description'] = df['game_description'].apply(lambda x: np.nan if x == ' About This Game Nothing' else x)
df['game_description'] = df['game_description'].apply(lambda x: np.nan if x == ' About This Game ...' else x)
df['game_description'] = df['game_description'].apply(lambda x: np.nan if x == ' About This Game ......' else x)
df['game_description'] = df['game_description'].apply(lambda x: np.nan if x == ' About This Game  YouTube: Chicharoney ' else x)
df['game_description'] = df['game_description'].apply(lambda x: np.nan if x == ' About This Content #auto_app_97368_about_default' else x)

#EXCLUINDO TODOS OS NAN DESSA COLUNA
df = df.dropna(subset=['game_description'])

# tratamento coluna release_date ->

In [42]:
# excluindo os valores nulos da coluna 
df = df.dropna(subset=['release_date'])

In [45]:
# aplicando a função para converter as datas em datetime e cria uma nova coluna 'data_formatada'
df['data formatada'] = df['release_date'].apply(transformando_datas_em_datatime)


In [47]:
# indentificando as linhas que possuem o nan 
mascara = df['data formatada'].isna()

# Substituir os valores nan na coluna 'data' pelos valores correspondentes da coluna 'url'
df.loc[mascara, 'data formatada'] = df.loc[mascara, 'url']

In [50]:
#pegar as urls q estão na coluna junto com as datas
lista_da_coluna_data_formatada = list(df['data formatada'])

#pegando as urls q estão na lista para fazer o web scraping
pegando_as_urls_da_coluna_data_formatada = [item for item in lista_da_coluna_data_formatada if isinstance(item, str)]
# pegando_as_urls_da_coluna_data_formatada

In [48]:
df[['release_date','data formatada']].sample(15)

,release_date,data formatada
4920,"Jun 7, 2016",2016-06-07 00:00:00
30119,"Mar 18, 2019",2019-03-18 00:00:00
25394,"Jun 9, 2018",2018-06-09 00:00:00
36715,Q3 2019,https://store.steampowered.com/app/1026420/WAR...
16534,"Oct 12, 2017",2017-10-12 00:00:00
24481,"Oct 16, 2018",2018-10-16 00:00:00
35679,"Jul 20, 2018",2018-07-20 00:00:00
14045,"Aug 16, 2013",2013-08-16 00:00:00
19599,"Feb 27, 2012",2012-02-27 00:00:00
21655,"Apr 5, 2017",2017-04-05 00:00:00


In [52]:
# fazer o web scraping com as urls já setadas
#lista com as datas de lançamento
lista_com_a_data_de_lacamento_dos_jogos = []
# lista com as urls manipulaveis
url_utilizavel_data = []
#lista com url sem uso
urls_sem_uso_data = []

url = pegando_as_urls_da_coluna_data_formatada
# criando um for para verificar cada url se da para usar ou nao
for i in url:  

    response = requests.get(i)
    
    # Verificar se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Parse a página HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # indo na class que contem a data 
        data_element = soup.find('div', class_='date')

        # Verificar se a data foi encontrada 
        if data_element:
            # pegando a data
            date_text = data_element.get_text().strip()

            lista_com_a_data_de_lacamento_dos_jogos.append(date_text)
            url_utilizavel_data.append(i)
        else:
            urls_sem_uso_data.append(i)
    else:
        urls_sem_uso_data.append(i)

In [54]:
df['data formatada'].unique()

array([Timestamp('2016-05-12 00:00:00'), Timestamp('2017-12-21 00:00:00'),
       Timestamp('2018-04-24 00:00:00'), ...,
       Timestamp('2019-09-12 00:00:00'),
       'https://store.steampowered.com/app/911510/Paws_and_Soul/',
       'https://store.steampowered.com/app/906470/Gravia/'], dtype=object)

In [55]:
#transformando as urls sem uso em nan
df.loc[df['data formatada'].isin(urls_sem_uso_data), 'data formatada'] = pd.NA

#dropando os nan da coluna
df = df.dropna(subset=['data formatada'])

In [56]:
df = df.reset_index()

In [57]:
# trocar as datas recebidas do web scraping pelas urls correspondente

# Inicialize um contador
contador = 0

# Loop para substituir as strings
# como a ordem bate
# quando o index da coluna for string ele vai pegar o valor da lista que é correspondendte e substituir
for i in range(len(df)):
    if isinstance(df.at[i, 'data formatada'], str) and contador < len(lista_com_a_data_de_lacamento_dos_jogos):
        df.at[i, 'data formatada'] = lista_com_a_data_de_lacamento_dos_jogos[contador]
        contador += 1

In [78]:
# verificando os valores que nao estão no formato de data
lista_da_coluna_data_formatada_valores_unicos = list(df['data formatada'].unique())
tentando_achar_valores_diferentes_de_data = [x for x in lista_da_coluna_data_formatada_valores_unicos if isinstance(x, str)]
verificando = [st for st in tentando_achar_valores_diferentes_de_data if len(st) < 12]
# verificando

In [79]:
# transformando em nan os valores q nao estao no formato de data
df['data formatada'] = df['data formatada'].apply(lambda x: np.nan if x == 'Coming soon' else x)

In [80]:
# #transformando novamente a coluna inteira em date time

# transformando todos os formato de data em data
df['data lancamento'] = df['data formatada'].apply(lambda x: parser.parse(x) if isinstance(x, str) else x)

In [81]:
df = df.dropna(subset=['data formatada'])

In [82]:
print(df.shape)
print(df.isna().sum())

(35810, 19)
level_0                 0
index                   0
url                     0
types                   0
name                    0
all_reviews          8184
release_date            0
developer             243
publisher            4771
popular_tags           98
game_details          435
languages               3
achievements        23849
genre                 347
game_description        0
price                   0
desconto                0
data formatada          0
data lancamento         0
dtype: int64


# tratando coluna achievements -> conquistas

In [83]:
# transformars os valores que estão como nan em url q sao correspondente

# indentificando as linhas que possuem o nan 
mascara = df['achievements'].isna()

# Substituir os valores nan na coluna 'data' pelos valores correspondentes da coluna 'url'
df.loc[mascara, 'achievements'] = df.loc[mascara, 'url']

In [84]:
# pegar as urls q estão na coluna 
# transformando toda a coluna achievements em uma lista
lista_da_coluna_achievements = list(df['achievements'])

#pegando as urls q estão na lista para fazer o web scraping
pegando_as_urls_da_coluna_achievements = [item for item in lista_da_coluna_achievements if isinstance(item, str)]
len(pegando_as_urls_da_coluna_achievements)

23849

In [85]:
# fazer o web scraping
lista_com_a_quantidade_de_conquista_de_cada_jogo = []
# lista com as urls manipulaveis
url_utilizavel_conquistas = []
#lista com url sem uso
urls_sem_uso_conquista = []

url = pegando_as_urls_da_coluna_achievements
# criando um for para verificar cada url se da para usar ou nao
for i in url:  

    response = requests.get(i)
    
    # Verificar se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Parse a página HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # indo na class que contem a conquista 
        data_element = soup.find('div', class_='communitylink_achievement_images')

        # Verificar se o elemento foi encontrado
        if data_element:
            # pegando a conquista
            date_text = data_element.get_text().strip()

            lista_com_a_quantidade_de_conquista_de_cada_jogo.append(date_text)
            url_utilizavel_conquistas.append(i)
        else:
            urls_sem_uso_conquista.append(i)
    else:
        urls_sem_uso_conquista.append(i)

In [135]:
print(len(lista_com_a_quantidade_de_conquista_de_cada_jogo))
print(len(url_utilizavel_conquistas))

1273
1273


In [117]:
# $ transformar os valores q estão vazio em suas urls q estão respectivas posições
# ao inves de vim o valor veio '' 

urls_q_veio_sem_valor_no_lugar_dos_valores_vazio = []

# percorrer os valores das duas listas
for conquista_item, url_item in zip(lista_com_a_quantidade_de_conquista_de_cada_jogo, url_utilizavel_conquistas):
    # verificar se o valor da lista "conquista" é igual a "''"
    if conquista_item == '':
        # se for igual vou adicionar o valor correspondente da lista "urls"
        urls_q_veio_sem_valor_no_lugar_dos_valores_vazio.append(url_item)
    else:
        # se nao vou manter o valor da lista "conquista"
        urls_q_veio_sem_valor_no_lugar_dos_valores_vazio.append(conquista_item)


In [136]:
len(urls_q_veio_sem_valor_no_lugar_dos_valores_vazio)
# len()

1273

In [137]:
pegando_as_urls = [st for st in urls_q_veio_sem_valor_no_lugar_dos_valores_vazio if len(st) > 30]
len(pegando_as_urls)

55

In [138]:
# transformei quase tudo em numero estava('Viewall 13') agora está('13') 
# menos as urls
lista_coom_valores_unitarios_das_conquistas = [percorrendo_lista_das_urls_que_substituiu_os_valores_vazio[8:] for percorrendo_lista_das_urls_que_substituiu_os_valores_vazio in urls_q_veio_sem_valor_no_lugar_dos_valores_vazio]
lista_coom_valores_unitarios_das_conquistas

# substituindo as urls pelo '999999999'
# Valor a ser substituído
valor_a_ser_trocado = '999999999'
# percorree a lista e substitui as URLs pelo valor '999999999'
for i in range(len(lista_coom_valores_unitarios_das_conquistas)):
    if lista_coom_valores_unitarios_das_conquistas[i].startswith('store.steampowered') or lista_coom_valores_unitarios_das_conquistas[i].startswith('store.steampowered'):
        lista_coom_valores_unitarios_das_conquistas[i] = valor_a_ser_trocado

#transformando todos os valores em float
lista_dos_valores_convertido_para_float = [float(valor) for valor in lista_coom_valores_unitarios_das_conquistas]
len(lista_dos_valores_convertido_para_float)

1273

In [139]:
len(urls_sem_uso_conquista)

22576

In [141]:
pegando_as_urls_da_coluna_achievements = [item for item in list(df['achievements']) if isinstance(item, str)]

len(pegando_as_urls_da_coluna_achievements)

23849

In [142]:
# criar uma nova coluna conquista
# se a url da lista sem uso for igual a da coluna achievements 
# na nova coluna esses valores serão 0 e mantendo os valores antigo

#mantive a função aq pq precisa da lista urls_sem_uso_conquista
def tranformando_url_sem_uso_em_0(valor):
    if valor in urls_sem_uso_conquista:
        return 0
    else:
        return valor

# Aplicar a função à coluna "conquista"
df['conquistas'] = df['achievements'].apply(tranformando_url_sem_uso_em_0)

In [143]:
pegando_as_urls_da_coluna_achievements = [item for item in list(df['conquistas']) if isinstance(item, str)]

len(pegando_as_urls_da_coluna_achievements)

1273

In [123]:
# df[['achievements','conquistas']].sample(15)

In [132]:
len(lista_dos_valores_convertido_para_float)

1273

In [146]:

# pegando_as_urls_da_coluna_achievements = [item for item in list(df['conquistas']) if isinstance(item, str)]

# pegando_as_urls_da_coluna_achievements

list(df['conquistas'])[31200:31204]

[0, 0, 0, 0]

In [148]:
df = df.drop(columns=['level_0', 'index'])


In [149]:
df = df.reset_index()

In [150]:
#com as urls q sobraram (as que teriam valor)]
# vai ser o valores correspondente de cada valor da lista lista_com_a_quantidade_de_conquista_de_cada_jogo
# então vou trocar as urls ---> valores da lista
contador = 0

# Loop para substituir as strings
for i in range(len(df)):
    if isinstance(df.at[i, 'conquistas'], str) and contador < len(lista_dos_valores_convertido_para_float):
        df.at[i, 'conquistas'] = lista_dos_valores_convertido_para_float[contador]
        contador += 1
df

,index,url,types,name,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,price,desconto,data formatada,data lancamento,conquistas
0,0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,"Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",19.99,75.55,2016-05-12 00:00:00,2016-05-12,54.0
1,1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,"Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,29.99,0.00,2017-12-21 00:00:00,2017-12-21,37.0
2,2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,"Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,39.99,0.00,2018-04-24 00:00:00,2018-04-24,128.0
3,3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,"Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,44.99,0.00,2018-12-13 00:00:00,2018-12-13,13.0
4,4,https://store.steampowered.com/app/601150/Devi...,app,Devil May Cry 5,"Very Positive,(9,645),- 92% of the 9,645 user ...","Mar 7, 2019","CAPCOM Co., Ltd.","CAPCOM Co., Ltd.,CAPCOM Co., Ltd.","Action,Hack and Slash,Great Soundtrack,Demons,...","Single-player,Online Multi-Player,Online Co-op...","English,French,Italian,German,Spanish - Spain,...",51.0,Action,About This Game The Devil you know returns in...,59.99,354.92,2019-03-07 00:00:00,2019-03-07,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35805,35806,https://store.steampowered.com/app/899836/Rock...,app,Rocksmith® 2014 Edition – Remastered – Sabaton...,NaN,"Feb 12, 2019",Ubisoft - San Francisco,NaN,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...",https://store.steampowered.com/app/899836/Rock...,"Casual,Simulation","About This Content Play ""Ghost Division"" by S...",2.99,0.00,2019-02-12 00:00:00,2019-02-12,0
35806,35807,https://store.steampowered.com/app/899832/Rock...,app,Rocksmith® 2014 Edition – Remastered – Stone T...,NaN,"Feb 5, 2019",Ubisoft - San Francisco,NaN,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...",https://store.steampowered.com/app/899832/Rock...,"Casual,Simulation","About This Content Play ""Trippin’ on a Hole i...",2.99,0.00,2019-02-05 00:00:00,2019-02-05,0
35807,35808,https://store.steampowered.com/app/906840/Fant...,app,Fantasy Grounds - Quests of Doom 4: A Midnight...,NaN,"Jul 31, 2018","SmiteWorks USA, LLC",NaN,"RPG,Indie,Strategy,Software,Turn-Based,Fantasy...","Multi-player,Co-op,Cross-Platform Multiplayer,...",English,https://store.steampowered.com/app/906840/Fant...,"Indie,RPG,Strategy",About This Content Quests of Doom 4: A Midni...,7.99,0.00,2018-07-31 00:00:00,2

In [151]:
df['conquistas'] = df['conquistas'].astype(int)

In [152]:
# os valores que tivem 999999999 vou cortar pq era os valores vazios

#transformando os valores 999999999 (as urls q nao conseguiram puxar o valor) em nan
df['conquistas'] = df['conquistas'].apply(lambda x: np.nan if x == 999999999 else x)

#excluindo os nan
df = df.dropna(subset=['conquistas'])

In [153]:
df['conquistas'] = df['conquistas'].astype(int)


C:\Users\Wellington\AppData\Local\Temp\ipykernel_12096\1288419239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['conquistas'] = df['conquistas'].astype(int)


# TRATAMENTO COLUNA publisher


In [154]:
df['publisher'] = df['publisher'].fillna('Sem Editora')


C:\Users\Wellington\AppData\Local\Temp\ipykernel_12096\465774373.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['publisher'] = df['publisher'].fillna('Sem Editora')


In [155]:
# transformando os valores suspeito em nan
df['publisher'] = df['publisher'].apply(lambda x: np.nan if x == ' , ' else x)
df['publisher'] = df['publisher'].apply(lambda x: np.nan if x == '--,--' else x)
df['publisher'] = df['publisher'].apply(lambda x: np.nan if x == '-,-' else x)

# excluindo os nan
df = df.dropna(subset=['publisher'])

C:\Users\Wellington\AppData\Local\Temp\ipykernel_12096\2599804063.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['publisher'] = df['publisher'].apply(lambda x: np.nan if x == ' , ' else x)
C:\Users\Wellington\AppData\Local\Temp\ipykernel_12096\2599804063.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['publisher'] = df['publisher'].apply(lambda x: np.nan if x == '--,--' else x)
C:\Users\Wellington\AppData\Local\Temp\ipykernel_12096\2599804063.py:4: SettingWithCopyWarning: 
A value is trying t

# TRATANDO COLUNA developer


In [156]:
# transformars os valores que estão como nan em url q sao correspondente

# indentificando as linhas que possuem o nan 
mascara = df['developer'].isna()

# substituir os valores nan na coluna 'developer' pelos valores correspondentes da coluna 'url'
df.loc[mascara, 'developer'] = df.loc[mascara, 'url']

# pegar as urls q estão na coluna developer
# transformando toda a coluna developer em uma lista
lista_da_coluna_developer = list(df['developer'])

# criando uma lista vazia onde vai vim as urls
lista_com_as_url_da_coluna_developer = []
# percorre a lista e substitui as URLs pelo valor desejado
for i in range(len(lista_da_coluna_developer)):
    #todos os valores que começa com 'https://store.steampowered' vai se guardado na lista ou seja todas as urls
    if lista_da_coluna_developer[i].startswith('https://store.steampowered') or lista_da_coluna_developer[i].startswith('https://store.steampowered'):
        lista_com_as_url_da_coluna_developer.append(lista_da_coluna_developer[i])
len(lista_com_as_url_da_coluna_developer)


243

In [157]:
# fazendo o web scraping
lista_com_a_desenvolvedora_de_cada_jogo = []
# lista com as urls manipulaveis
url_utilizavel_desenvolvedora = []
#lista com url sem uso
urls_sem_uso_desenvolvedora = [] 

url = lista_com_as_url_da_coluna_developer
# criando um for para verificar cada url se da para usar ou nao
for i in url:  

    response = requests.get(i)
    
    # Verificar se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Parse a página HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar o elemento que contém a data de lançamento
        data_element = soup.find('div', class_='dev_row')

        # Verificar se o elemento foi encontrado
        if data_element:
            # Extrair o texto do elemento
            date_text = data_element.get_text().strip()

            lista_com_a_desenvolvedora_de_cada_jogo.append(date_text)
            url_utilizavel_desenvolvedora.append(i)
        else:
            urls_sem_uso_desenvolvedora.append(i)
    else:
        urls_sem_uso_desenvolvedora.append(i)

In [158]:
# criar uma nova coluna 'desenvolvedora'
# se a url da lista 'urls_sem_uso_desenvolvedora_obs' for igual a da coluna developer 
# na nova coluna esses valores serão nan e mantendo os valores antigo

def transformando_em_nan_as_urls_sem_uso(valor):
    if valor in urls_sem_uso_desenvolvedora:
        return np.nan
    else:
        return valor

# aplicar a função à coluna "developer"
df['desenvolvedora'] = df['developer'].apply(transformando_em_nan_as_urls_sem_uso)

In [159]:
#excluuir os nan da coluna
df = df.dropna(subset=['desenvolvedora'])

In [160]:
# transformar as strings que não começam com 'Developer:' em nan
for i in range(len(lista_com_a_desenvolvedora_de_cada_jogo)):
    if not lista_com_a_desenvolvedora_de_cada_jogo[i].startswith('Developer:'):
        lista_com_a_desenvolvedora_de_cada_jogo[i] = np.nan
    else:
        lista_com_a_desenvolvedora_de_cada_jogo[i] = lista_com_a_desenvolvedora_de_cada_jogo[i].replace('Developer:\n\n', '')


In [161]:
df = df.reset_index()

In [162]:
#verificar se são todas str
df['desenvolvedora'].unique()

array(['id Software', 'PUBG Corporation', 'Harebrained Schemes', ...,
       'Dan Goyette', 'Gustavo Contreras', '16 Bit Psych,Kyle B'],
      dtype=object)

In [163]:
len(lista_com_a_desenvolvedora_de_cada_jogo)

94

In [164]:
# Função para substituir URLs por valores da lista
def substituir_url_pelo_valor_da_lista(valor):
    if valor.startswith('https://store.steampowered.com/app/') and len(lista_com_a_desenvolvedora_de_cada_jogo) > 0:
        return lista_com_a_desenvolvedora_de_cada_jogo.pop(0)
    return valor

# Aplicar a função à coluna 'preço'
df['desenvolvedora'] = df['desenvolvedora'].apply(substituir_url_pelo_valor_da_lista)

In [165]:
df = df.dropna(subset=['desenvolvedora'])

# TRATANDO COLUNA all_reviews


In [167]:
df.shape

(35496, 21)

In [168]:
df = df.dropna(subset=['all_reviews'])

In [169]:
# tranformando a coluna em lista (valores antigos e as urls)
# vou manter as urls e alterar os valores antigos  
# Mixed,(4,598),- 62% of the 4,598 user reviews for this ----> 92% of the 42,550 user reviews for this
# vou colocar esses valores formatados + urls em uma nova coluna
# criar uma nova coluna 'manipulando review'

segunda_lista_com__review_geral = list(df['all_reviews'])
# Função para extrair a parte desejada  
def recortando_a_lista_mantendo_url(text):
    if 'http' in text:
        return text  # Mantém URLs inalteradas
    return text.split('- ')[1].strip()

# Aplica a função para extrair a parte desejada nas strings
lista_com_os_valores_antigos_manipulados = [recortando_a_lista_mantendo_url(text) for text in segunda_lista_com__review_geral]
df['manipulando_review'] = lista_com_os_valores_antigos_manipulados

In [170]:
df['manipulando_review'] = df['manipulando_review'].apply(lambda x: 'Sem avaliacao' if x == 'Need more user reviews to generate a score' else x)

In [171]:
df = df.dropna(subset=['genre', 'languages', 'game_details', 'popular_tags'])

In [212]:
print(df.shape)
print(df.isna().sum())

(26963, 22)
level_0               0
index                 0
url                   0
types                 0
name                  0
all_reviews           0
release_date          0
developer             0
publisher             0
popular_tags          0
game_details          0
languages             0
achievements          0
genre                 0
game_description      0
price                 0
desconto              0
data formatada        0
data lancamento       0
conquistas            0
desenvolvedora        0
manipulando_review    0
dtype: int64


In [214]:
df = df.drop(columns=['level_0', 'index', 'release_date', 'data formatada', 'developer', 'achievements'])
df = df.rename(columns={'manipulando_review': 'avaliação geral'})

In [215]:
print(df.shape)
print(df.isna().sum())

(26963, 16)
url                 0
types               0
name                0
all_reviews         0
publisher           0
popular_tags        0
game_details        0
languages           0
genre               0
game_description    0
price               0
desconto            0
data lancamento     0
conquistas          0
desenvolvedora      0
avaliação geral     0
dtype: int64


In [216]:
df.to_csv('Steam_games_tratada.csv')

pivot coluna genre

In [218]:
# passando as colunas genre para outro DF
pivot_genre = df['genre']

# transformando em uma lista
tag_lists = [tags.split(',') for tags in pivot_genre]
# tag_lists

# # Crie um DataFrame com uma coluna 'genre' em formato de lista
df_pivot_genre = pd.DataFrame({'genre': tag_lists})
# trouxe a coluna url para esse novo df para ser nosso id
df_pivot_genre['url'] = df['url']
# df_pivot_genre

#fazendo o pivotamento (nao sei como escreve)
df_pivot_genre_feito = df_pivot_genre.explode('genre')

#criando um csv aparte para o pivot
df_pivot_genre_feito.to_csv('steam_games_pivotamento_genre.csv')

pivot coluna languages

In [219]:
# passando as colunas genre para outro DF
pivot_languages = df['languages']

# transformando em uma lista
tag_lists = [tags.split(',') for tags in pivot_languages]
# tag_lists

# # Crie um DataFrame com uma coluna 'genre' em formato de lista
df_pivot_languages = pd.DataFrame({'languages': tag_lists})
# trouxe a coluna url para esse novo df para ser nosso id
df_pivot_languages['url'] = df['url']

#fazendo o pivotamento (nao sei como escreve)
df_pivot_languages_feito = df_pivot_languages.explode('languages')

#criando um csv aparte para o pivot
df_pivot_languages_feito.to_csv('steam_games_pivotamento_languages.csv')

pivot game_details

In [220]:
# passando as colunas game_details para outro DF
pivot_game_details = df['game_details']

# transformando em uma lista
tag_lists = [tags.split(',') for tags in pivot_game_details]

# # Crie um DataFrame com uma coluna 'game_details' em formato de lista
df_pivot_game_details = pd.DataFrame({'game_details': tag_lists})
# trouxe a coluna url para esse novo df para ser nosso id
df_pivot_game_details['url'] = df['url']

#fazendo o pivotamento (nao sei como escreve)
df_pivot_game_details_feito = df_pivot_game_details.explode('game_details')

#criando um csv aparte para o pivot
df_pivot_game_details_feito.to_csv('steam_games_pivotamento_game_details.csv')

pivot popular_tags

In [221]:
# passando as colunas game_details para outro DF
pivot_popular_tags = df['popular_tags']

# transformando em uma lista
tag_lists = [tags.split(',') for tags in pivot_popular_tags]

# # Crie um DataFrame com uma coluna 'popular_tags' em formato de lista
df_pivot_popular_tags = pd.DataFrame({'popular_tags': tag_lists})
# trouxe a coluna url para esse novo df para ser nosso id
df_pivot_popular_tags['url'] = df['url']

#fazendo o pivotamento (nao sei como escreve)
df_pivot_popular_tags_feito = df_pivot_popular_tags.explode('popular_tags')

#criando um csv aparte para o pivot
df_pivot_popular_tags_feito.to_csv('steam_games_pivotamento_popular_tags.csv')